In [1]:
# Use this script to save csv files into database with their filename as tablename
import pandas as pd
import os
from sqlalchemy import create_engine
import logging
import time

# Configure logging
logging.basicConfig(
    filename="logs/ingestion_db.log", 
    level=logging.DEBUG,
    format="%(asctime)s - %(levelname)s - %(message)s", 
    filemode="a"  
)

# Path to MY CSV folder
DATA_PATH = r"C:\Users\Manish\OneDrive\Desktop\Vendor Performance Analysis\data"

# Create SQLite database connection
engine = create_engine('sqlite:///inventory.db')

def ingest_db(df, table_name, engine):
    """Ingests the dataframe into database table"""
    df.to_sql(table_name, con=engine, if_exists='replace', index=False)

def load_raw_data():
    """Loads all CSVs as dataframe and ingests into db"""
    start = time.time()
    
    if not os.path.exists(DATA_PATH):
        logging.error(f"Data folder not found: {DATA_PATH}")
        return
    
    for file in os.listdir(DATA_PATH):
        if file.endswith('.csv'):
            file_path = os.path.join(DATA_PATH, file)
            try:
                df = pd.read_csv(file_path)
                logging.info(f'Ingesting {file} into db')
                ingest_db(df, file[:-4], engine)
            except Exception as e:
                logging.error(f"Error processing {file}: {str(e)}")
    
    end = time.time()
    total_time = (end - start)/60
    logging.info('--------------Ingestion Complete------------')
    logging.info(f'Total Time Taken: {total_time:.2f} minutes')

if __name__ == '__main__':
    load_raw_data()
